In [3]:
import numpy as np
import glob
import os 
import matplotlib.pyplot as plt
from scipy.stats import zscore
import seaborn as sns
import pandas as pd

In [4]:
def getChi(data):
    N = data.shape[0]
    t_max = data.shape[1]
    cov = 1/t_max * (data @ data.T) # NxN
    # print(cov.shape)
    mu = np.mean(data, axis=1) # Nx1
    m_ij = np.outer(mu, mu)
    # print(m_ij.shape)
    c = cov - m_ij # NxN
    # print(np.sum(cov))
    chiSG = np.sum(c**2, axis=1)
    chiUni = np.sum(c, axis=1)
    # l, _ = np.linalg.eig(cov)
    # chiSG = np.dot(l, l) / N;
    # print(chiSG, chiUni)
    return chiSG, chiUni, c, cov, mu

In [5]:
z_ts_files = glob.glob('data/input_data/abide/nyu_aal/*1D')
timeseries = [np.loadtxt(i) for i in z_ts_files]
timeseries = np.array(timeseries)
meta = pd.read_csv('data/metadata.csv')
print(timeseries.shape)
print(timeseries[0].shape)

(171,)
(176, 116)


/tmp/ipykernel_1937/3088241977.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  timeseries = np.array(timeseries)


In [6]:
datalen = [i.shape[0] for i in timeseries]
datalen = np.array(datalen)
t_max = range(130, 145, 1)
count = []
for t in t_max:
    c = np.sum(datalen > t)
    count.append(c)
# plt.plot(t_max, count)
t_max = t_max[::-1]
count = count[::-1]
z = np.array([i*j  for i, j in zip(t_max, count)])
# print(list(t_max))
# print(count)
count = np.array(count)
idx = np.argmax(z)
print('t_max:', t_max[idx])
print('Number of subjects remaining:', count[idx])
max_t = t_max[idx]

t_max: 137
Number of subjects remaining: 145


In [13]:
rois = timeseries[0].shape[1]
data_trunc = [] 
thresholds = {}

subids = []
for i in z_ts_files: 
    subid = os.path.basename(i).split('_')[1]
    subids.append(int(subid))
subid

'0051156'

In [17]:
meta[meta['SUB_ID'] == 51156]['DX_GROUP'].iloc[0] == 1

False

In [18]:
trunc_subids = []
cn_timeseries = []
asd_timeseries = []
for idx, i in enumerate(timeseries):
    if i.shape[0] < max_t or i.shape[1] != rois:
        print('skipping', subids[idx])
        continue
    trunc_subids.append(subids[idx])
    diag = meta[meta['SUB_ID'] == subids[idx]]['DX_GROUP'].iloc[0]
    trunc = i[:max_t, :]
    trunc = trunc - np.mean(trunc, axis=0)
    s_max = np.max(trunc)
    s_min = np.min(trunc)
    # print(s_max, s_min)
    # point at which the line connecting (s_min, -1) and (s_max, 1) meets x-axis
    thres = (s_max + s_min) / 2
    thresholds[subids[idx]] = thres
    # plt.plot(trunc[:, 0])
    # plt.axhline(y=thres, color='black', linestyle='--')
    # plt.show()
    # print(thres)
    binarized = np.where(trunc > thres, 1, -1)
    # print(binarized.shape)
    data_trunc.append(binarized.T) 
    if diag == 1:
        asd_timeseries.append(binarized.T)
    else:
        cn_timeseries.append(binarized.T)
    # plt.plot(data_trunc[-1][0])
    # plt.show()
#     data_trunc.append(trunc) 
data_trunc = np.array(data_trunc)
cn_timeseries = np.array(cn_timeseries)
asd_timeseries = np.array(asd_timeseries)    
print(data_trunc.shape)
print(cn_timeseries.shape)
print(asd_timeseries.shape)

skipping 50957
skipping 50965
skipping 50969
skipping 50979
skipping 50984
skipping 50997
skipping 50998
skipping 51002
skipping 51008
skipping 51011
skipping 51012
skipping 51013
skipping 51020
skipping 51033
skipping 51034
skipping 51035
skipping 51045
skipping 51060
skipping 51077
skipping 51082
skipping 51083
skipping 51087
skipping 51093
skipping 51123
skipping 51150
(146, 116, 137)
(89, 116, 137)
(57, 116, 137)


In [19]:
data_concat = np.concatenate(data_trunc, axis=1)
cn_concat = np.concatenate(cn_timeseries, axis=1)
asd_concat = np.concatenate(asd_timeseries, axis=1)
print(data_concat.shape)
print(cn_concat.shape)
np.savetxt('data/abide_nyu_cn_concat.txt', cn_concat)

(116, 20002)
(116, 12193)
